In [ ]:
Algorithm (static)

In [3]:
import open3d as o3d
import numpy as np

# Function to load KITTI .ply file
def load_kitti_ply(file_path):
    """Load KITTI point cloud from .ply file."""
    pcd = o3d.io.read_point_cloud(file_path)  # Read the .ply file
    print(f"Loaded {len(pcd.points)} points from {file_path}")
    return pcd

# Function to assign a uniform color to a point cloud
def assign_color(cloud, color):
    cloud.paint_uniform_color(color)

# Function to determine if a cluster is a tree
def is_tree(cluster):
    bbox = cluster.get_axis_aligned_bounding_box()
    size = bbox.get_extent()
    height_to_width = size[2] / max(size[0], size[1])
    return height_to_width > 0.35 and size[2] > 1.0  # Tall and slim clusters

# Function to determine if a cluster is a building
def is_building(cluster):
    bbox = cluster.get_axis_aligned_bounding_box()
    size = bbox.get_extent()
    return size[2] > 3.0  # Tall and rectangular

# Function to determine if a cluster is a vehicle
def is_vehicle(cluster):
    bbox = cluster.get_axis_aligned_bounding_box()
    size = bbox.get_extent()
    return 1.5 <= size[0] <= 4.0 and 1.5 <= size[1] <= 4.0 and 1.0 <= size[2] <= 3.0

# Function to classify and visualize buildings, trees, planes, and vehicles
def classify_and_visualize(pcd, output_file_path="output_segmented.ply"):
    # Step 1: Segment the largest plane (road)
    plane_model, road_cloud, remaining_cloud = segment_plane(pcd, distance_threshold=0.3)
    assign_color(road_cloud, [0.6, 0.6, 0.6])  # Grey for road
    print("Road segmented.")

    # Step 2: Cluster remaining points
    labels = np.array(remaining_cloud.cluster_dbscan(eps=0.5, min_points=10, print_progress=True))
    print(f"Found {labels.max() + 1} clusters.")

    cluster_clouds = []
    undefined_clouds = []
    total_points = len(pcd.points)
    road_points = len(road_cloud.points)
    tree_points = 0
    building_points = 0
    vehicle_points = 0
    undefined_points = 0

    for i in range(labels.max() + 1):
        cluster = remaining_cloud.select_by_index(np.where(labels == i)[0])

        # Classify clusters
        if is_tree(cluster):
            assign_color(cluster, [0, 1, 0])  # Green for trees
            cluster_clouds.append(cluster)
            tree_points += len(cluster.points)
        elif is_building(cluster):
            assign_color(cluster, [0, 0, 1])  # Blue for buildings
            cluster_clouds.append(cluster)
            building_points += len(cluster.points)
        elif is_vehicle(cluster):
            assign_color(cluster, [1, 0, 0])  # Red for vehicles
            cluster_clouds.append(cluster)
            vehicle_points += len(cluster.points)
        else:
            assign_color(cluster, [1, 1, 1])  # White for undefined
            undefined_clouds.append(cluster)
            undefined_points += len(cluster.points)

    # Combine all clouds
    all_clouds = [road_cloud] + cluster_clouds + undefined_clouds
    combined_cloud = all_clouds[0]
    for cloud in all_clouds[1:]:
        combined_cloud += cloud

    # Save the combined point cloud to .ply
    o3d.io.write_point_cloud(output_file_path, combined_cloud)
    print(f"Segmented point cloud saved to {output_file_path}")

    # Visualize results
    print("Classification complete. Visualizing...")
    o3d.visualization.draw_geometries(all_clouds)

    # Print the number of points in each segment
    print(f"Total points: {total_points}")
    print(f"Road points: {road_points}")
    print(f"Tree points: {tree_points}")
    print(f"Building points: {building_points}")
    print(f"Vehicle points: {vehicle_points}")
    print(f"Undefined points: {undefined_points}")

# Function to segment the largest plane in the point cloud
def segment_plane(pcd, distance_threshold=0.3, ransac_n=3, num_iterations=1000):
    """Segment the largest plane in the point cloud."""
    plane_model, inliers = pcd.segment_plane(distance_threshold=distance_threshold,
                                             ransac_n=ransac_n,
                                             num_iterations=num_iterations)
    inlier_cloud = pcd.select_by_index(inliers)
    outlier_cloud = pcd.select_by_index(inliers, invert=True)
    return plane_model, inlier_cloud, outlier_cloud

# Specify file paths
file_path = "C:/Users/rr318/Desktop/minifinal/0000003106_0000003313.ply"
output_file_path = "C:/Users/rr318/Desktop/31output_segmented.ply"

# Load point cloud and classify
pcd = load_kitti_ply(file_path)
classify_and_visualize(pcd, output_file_path)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Loaded 2122609 points from C:/Users/rr318/Desktop/minifinal/0000003106_0000003313.ply
Road segmented.
Found 991 clusters.
Segmented point cloud saved to C:/Users/rr318/Desktop/31output_segmented.ply
Classification complete. Visualizing...
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
Total points: 2122609
Road points: 1292041
Tree points: 312649
Building points: 330507
Vehicle points: 16915
Undefined points: 150680


In [ ]:
Evaluation Algorithm 

In [1]:
import open3d as o3d
import numpy as np
from scipy.spatial import KDTree

# Load point cloud from a PLY file
def load_point_cloud(file_path):
    """
    Load a PLY point cloud file.
    Returns an Nx6 numpy array with XYZRGB values.
    """
    pcd = o3d.io.read_point_cloud(file_path)
    points = np.asarray(pcd.points)  # Nx3 array of points (x, y, z)
    colors = np.asarray(pcd.colors)  # Nx3 array of colors (r, g, b)
    return np.hstack((points, colors))  # Nx6 array of XYZRGB

# Color to Class Mapping based on RGB values
def classify_point(point_rgb):
    """
    Classify a point based on its RGB value into one of 5 classes:
    - Green: Trees
    - Red: Vehicles
    - Blue: Buildings
    - White: Unidentified
    - Grey: Road/Plane
    """
    if np.allclose(point_rgb, [0, 1, 0], atol=0.1):  # Green for Trees
        return "Trees"
    elif np.allclose(point_rgb, [1, 0, 0], atol=0.1):  # Red for Vehicles
        return "Vehicles"
    elif np.allclose(point_rgb, [0, 0, 1], atol=0.1):  # Blue for Buildings
        return "Buildings"
    elif np.allclose(point_rgb, [1, 1, 1], atol=0.1):  # White for Unidentified
        return "Unidentified"
    elif np.allclose(point_rgb, [0.5, 0.5, 0.5], atol=0.1):  # Grey for Road/Plane
        return "Road/Plane"
    else:
        return "Unknown"  # For any other color (if needed)

# Compare point clouds with RGB properties and match based on color class
def compare_point_clouds_by_class(pc1, pc2, distance_threshold=0.1):
    """
    Compare two point clouds using RGB properties and match points based on color classes.
    Args:
        pc1: First point cloud (Nx6 numpy array).
        pc2: Second point cloud (Mx6 numpy array).
        distance_threshold: Max distance to consider a match.
    Returns:
        metrics_by_class: Dictionary containing comparison metrics for each class.
        ignored_points_count: Number of points in pc1 without a valid match in pc2.
    """
    # Separate spatial and color data
    coords1, rgb1 = pc1[:, :3], pc1[:, 3:]
    coords2, rgb2 = pc2[:, :3], pc2[:, 3:]

    # Build a KDTree for nearest-neighbor search
    tree = KDTree(coords2)
    distances, indices = tree.query(coords1)

    # Find valid matches within the distance threshold
    valid_matches = distances <= distance_threshold
    ignored_points_count = np.sum(~valid_matches)

    # Create a dictionary to store results per class
    metrics_by_class = {
        "Trees": {"matched": 0, "ignored": 0},
        "Vehicles": {"matched": 0, "ignored": 0},
        "Buildings": {"matched": 0, "ignored": 0},
        "Unidentified": {"matched": 0, "ignored": 0},
        "Road/Plane": {"matched": 0, "ignored": 0}
    }

    # Iterate through matched points and classify by color
    for i, (valid, dist, idx) in enumerate(zip(valid_matches, distances, indices)):
        if valid:  # Only process valid matches
            class1 = classify_point(rgb1[i])
            class2 = classify_point(rgb2[idx])

            # If both points belong to the same class, count the match
            if class1 == class2:
                metrics_by_class[class1]["matched"] += 1
            else:
                # Ignore mismatched classes
                metrics_by_class[class1]["ignored"] += 1
                metrics_by_class[class2]["ignored"] += 1

    return metrics_by_class, ignored_points_count

# Example usage
file1 = "C:/Users/rr318/Desktop/minifinal/gt_pointsize.ply"  # Replace with your file path
file2 = "C:/Users/rr318/Desktop/minifinal/algo_output.ply"  # Replace with your file path
pc1 = load_point_cloud(file1)
pc2 = load_point_cloud(file2)

distance_threshold = 0.1  # Max distance for matching
metrics_by_class, ignored_points = compare_point_clouds_by_class(pc1, pc2, distance_threshold)

# Print results
print("Comparison Metrics by Class:")
for class_name, metrics in metrics_by_class.items():
    print(f"Class: {class_name}")
    print(f"  Matched Points: {metrics['matched']}")
    print(f"  Ignored Points: {metrics['ignored']}")
print(f"\nTotal Ignored Points: {ignored_points}")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Comparison Metrics by Class:
Class: Trees
  Matched Points: 105714
  Ignored Points: 265859
Class: Vehicles
  Matched Points: 15581
  Ignored Points: 125099
Class: Buildings
  Matched Points: 275887
  Ignored Points: 244950
Class: Unidentified
  Matched Points: 17714
  Ignored Points: 258791
Class: Road/Plane
  Matched Points: 1159496
  Ignored Points: 162437

Total Ignored Points: 19649


In [2]:
def calculate_accuracy(metrics_by_class, pc1):
    """
    Calculate the accuracy for each class by dividing the matched points by the total points
    in that class in the first point cloud (pc1), then multiplying by 100 to get the percentage.
    """
    total_points_by_class = {
        "Trees": np.sum([np.allclose(rgb, [0, 1, 0], atol=0.1) for rgb in pc1[:, 3:]]),
        "Vehicles": np.sum([np.allclose(rgb, [1, 0, 0], atol=0.1) for rgb in pc1[:, 3:]]),
        "Buildings": np.sum([np.allclose(rgb, [0, 0, 1], atol=0.1) for rgb in pc1[:, 3:]]),
        "Unidentified": np.sum([np.allclose(rgb, [1, 1, 1], atol=0.1) for rgb in pc1[:, 3:]]),
        "Road/Plane": np.sum([np.allclose(rgb, [0.5, 0.5, 0.5], atol=0.1) for rgb in pc1[:, 3:]])
    }

    accuracy_by_class = {}
    for class_name in metrics_by_class:
        matched_points = metrics_by_class[class_name]["matched"]
        total_points = total_points_by_class.get(class_name, 0)
        
        if total_points > 0:
            accuracy = (matched_points / total_points) * 100
        else:
            accuracy = 0  # If there are no points in the class, accuracy is 0
        
        accuracy_by_class[class_name] = accuracy
    
    return accuracy_by_class

# Example usage to calculate accuracy
accuracy_by_class = calculate_accuracy(metrics_by_class, pc1)

# Print the accuracy for each class
print("Accuracy by Class:")
for class_name, accuracy in accuracy_by_class.items():
    print(f"Class: {class_name}, Accuracy: {accuracy:.2f}%")

Accuracy by Class:
Class: Trees, Accuracy: 59.14%
Class: Vehicles, Accuracy: 11.18%
Class: Buildings, Accuracy: 58.56%
Class: Unidentified, Accuracy: 12.31%
Class: Road/Plane, Accuracy: 97.49%


In [4]:
def calculate_total_efficiency_by_average(accuracy_by_class):
    """
    Calculate the total efficiency by averaging the accuracies of all classes,
    excluding the "Unidentified" class.
    """
    total_accuracy = 0
    num_classes = 0
    
    for class_name, accuracy in accuracy_by_class.items():
        if class_name != "Unidentified":  # Skip the "Unidentified" class
            total_accuracy += accuracy
            num_classes += 1
    
    # Calculate the average efficiency (total efficiency)
    if num_classes > 0:
        total_efficiency = total_accuracy / num_classes
    else:
        total_efficiency = 0  # If there are no valid classes, efficiency is 0
    
    return total_efficiency

# Example usage to calculate total efficiency by averaging class accuracies
total_efficiency = calculate_total_efficiency_by_average(accuracy_by_class)

# Print the total efficiency
print(f"Total Efficiency (excluding unidentified): {total_efficiency:.2f}%")


Total Efficiency (excluding unidentified): 56.59%


In [ ]:
import open3d as o3d

# Function to count the total number of points in a .ply file
def count_points_in_ply(file_path):
    """
    Count the total number of points in a .ply file.
    
    Args:
        file_path (str): Path to the .ply file.
    
    Returns:
        int: Total number of points in the point cloud.
    """
    # Load the point cloud
    point_cloud = o3d.io.read_point_cloud(file_path)
    
    # Get the number of points
    total_points = len(point_cloud.points)
    return total_points

# Example usage
file_path = "path/to/your/point_cloud.ply"  # Replace with your .ply file path
total_points = count_points_in_ply(file_path)
print(f"Total number of points in the .ply file: {total_points}")
